In [2]:
import warnings
warnings.filterwarnings("ignore")
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json
import datetime
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
import joblib

### Load model :

In [3]:
model=load_model('deep_model.h5', compile=False)

2025-05-29 15:05:52.172684: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
df=pd.read_csv('no_outliers.csv')
df

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,0.946187,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,429.332315,0.133511
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,0.910936,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,194.210750,0.363279
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,0.896924,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,648.404343,0.280041
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,0.869940,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,885.138727,0.125099
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,0.878946,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,602.697524,0.320636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,0.668902,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,534.140819,0.364438
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,0.818049,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,387.264753,0.564494
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,0.896188,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,476.450471,0.591383
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,0.860886,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1015.811664,-0.093105


In [5]:
scaler=joblib.load('scaler_SS_DART.pkl')
encoder=joblib.load('encoder_OH_DART.pkl')
bins_a=np.load('mprime_a_bins_DART.npy')
bins_g=np.load('mprime_g_bins_DART.npy')

In [6]:
df['mprime_g_bin'] = pd.cut(df['mprime_g'], bins=bins_g, labels=False, include_lowest=True)
df['mprime_a_bin'] = pd.cut(df['mprime_a'], bins=bins_a, labels=False, include_lowest=True)

In [7]:
cols_to_scale = df.drop(columns=['alpha','BOA_RT','mprime_a_bin','mprime_g_bin']).columns
cols_to_encode = ['mprime_a_bin', 'mprime_g_bin']

In [8]:
X_scaled = scaler.transform(df[cols_to_scale])

X_bins_encoded = encoder.transform(df[cols_to_encode])

# Récupération des noms des colonnes encodées
encoded_feature_names = encoder.get_feature_names_out(cols_to_encode)

# Construction du DataFrame final
df_scaled = pd.DataFrame(
    np.concatenate([X_scaled, X_bins_encoded], axis=1),
    columns=list(cols_to_scale) + list(encoded_feature_names),
    index=df.index  # Pour conserver les bons index
)


In [9]:
df_scaled['BOA_RT']=df['BOA_RT']
df_scaled['alpha']=df['alpha']
df_scaled

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4,BOA_RT,alpha
0,0.938938,0.830099,-0.143116,0.205274,-0.748201,-0.102195,-0.148214,1.494961,-0.196407,-0.449844,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,429.332315,0.133511
1,0.072507,-1.551574,1.189851,0.880461,-0.191308,-0.932700,-0.652588,1.032032,1.082365,-0.470705,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,194.210750,0.363279
2,-2.603383,0.858439,-0.191497,0.141423,2.954771,-0.107965,-0.172161,-1.032376,-0.241683,0.598065,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,648.404343,0.280041
3,0.181490,0.807331,1.187661,-1.173034,-0.267672,-1.155707,-0.908316,0.025438,-1.062994,0.958397,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,885.138727,0.125099
4,-0.956225,0.171426,-0.715124,0.448112,0.653581,0.622586,0.469534,-0.757894,-0.363971,0.962627,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,602.697524,0.320636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,-0.142545,-0.840552,1.094456,1.179116,-0.034307,-0.646551,-0.355299,-0.818006,-0.979054,0.834325,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,534.140819,0.364438
362499,-0.217888,-0.025624,-0.220377,0.728064,0.022821,0.496683,0.508949,0.483711,-0.757920,-1.677835,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,387.264753,0.564494
362500,-1.260520,-0.089295,-0.383105,0.765189,0.960787,0.743714,0.732254,-1.475407,-0.782331,-1.559563,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,476.450471,0.591383
362501,-0.789749,0.950134,1.119665,0.483449,0.498611,-0.969059,-0.716191,-1.421001,0.517531,1.153519,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1015.811664,-0.093105


In [10]:
X=df_scaled.drop(columns=['alpha'])
y=df_scaled['alpha']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
BOA_train=X_train['BOA_RT']
BOA_test=X_test['BOA_RT']
X_train.drop(columns=['BOA_RT'],inplace=True)
X_test.drop(columns=['BOA_RT'],inplace=True)

In [11]:
y_pred=model.predict(X_test)

2833/2833 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [12]:
y_test

159680   -0.224256
347342    0.376680
307069    0.238902
126659    0.471005
356896    0.128247
            ...   
326002    0.194530
190999   -0.142424
27703     0.149952
76624     0.143431
189606    0.229081
Name: alpha, Length: 90626, dtype: float64

In [13]:
y_pred

array([[-0.21968824],
       [ 0.3766121 ],
       [ 0.22941366],
       ...,
       [ 0.1528129 ],
       [ 0.14012328],
       [ 0.23602526]], dtype=float32)

In [15]:
MAE = mean_absolute_error(y_test, y_pred)
MAE

0.004759879721172527

In [16]:
X_train

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_0,mprime_a_bin_1,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4
80661,-0.069120,0.880779,0.640769,-0.173369,-0.088895,-0.790171,-0.665889,1.520224,0.279489,1.004118,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
248726,-1.330278,-2.773945,-0.221436,-0.573381,1.036162,-0.425127,-0.545665,-1.069706,1.448924,0.855865,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
247518,-0.447697,-1.204405,1.493352,-0.202837,0.204542,-1.268209,-0.942720,0.247516,0.436509,-0.688671,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
303607,-1.111384,0.874273,0.948692,-1.774713,0.806064,-1.098430,-0.918408,0.994639,-0.978477,-0.442122,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
305103,-3.228436,0.674241,-1.167519,0.118035,4.781522,0.640134,0.312535,-0.022583,-0.726047,1.270408,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.752914,0.744033,0.041159,-1.205458,-0.637513,-0.706761,-0.774369,-0.775916,0.129560,0.250184,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
259178,0.930569,0.612489,-1.441607,0.123058,-0.743313,0.876728,0.471052,-1.627810,-0.521755,-1.252927,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
131932,0.923727,0.498173,-0.628089,-1.309832,-0.739311,-0.445585,-0.719076,-0.930604,0.217773,-0.864780,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
146867,0.379052,0.323402,0.139977,0.378624,-0.401067,-0.210751,-0.172633,1.146767,0.084081,1.734614,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
cols_to_scale_inv = X_train.drop(columns=['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4'
       ]).columns
cols_to_encode_inv = ['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4',
       ]

In [22]:
X_scaled_part = X_test[cols_to_scale_inv].values
X_encoded_part = X_test[cols_to_encode_inv].values
# 2. Inverser la standardisation
X_original_scaled = scaler.inverse_transform(X_scaled_part)

# 3. Inverser l'encodage
X_original_categoricals = encoder.inverse_transform(X_encoded_part)

In [23]:
X_test_real= pd.DataFrame(
    X_original_scaled,
    columns=['Tg_scat', 'Tg_abs', 'Ta_abs', 'SSA', 'GOD', 'AOD', 'AODS', 'SZA', 'Z',
       'R_scence', 'g1', 'Cos(SZA)', 'mu_g', 'mu_a', 'muprime_g', 'muprime_a',
       'mprime_g', 'mprime_a'],
    index=X_test.index
)
df_categoricals = pd.DataFrame(
    X_original_categoricals,
    columns=['mprime_a_bin', 'mprime_g_bin'],
    index=X_test.index
)
X_test_real = pd.concat([X_test_real, df_categoricals], axis=1)

In [24]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,0.849374,0.951706,708.317583,3187.429121,0.951776,0.951721,0.684362,0.152645,0,0
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,0.814209,0.470531,708.012549,3186.056472,0.471695,0.470791,1.873413,1.217586,0,0
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,0.915442,0.240763,707.890641,3185.507886,0.243492,0.241375,4.099723,4.110381,0,0
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,0.852120,0.453122,708.341159,3187.535216,0.454355,0.453397,1.400187,0.288163,0,0
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,0.869006,0.999158,707.958273,3185.812230,0.999159,0.999158,0.933753,0.732430,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,0.959487,0.962254,708.064025,3186.288112,0.962308,0.962266,0.872219,0.472533,0,0
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,0.812618,0.025776,708.122465,3186.551091,0.042399,0.030856,18.672395,11.328534,0,2
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,0.789790,0.909007,707.894025,3185.523112,0.909141,0.909037,1.094304,1.074933,0,0
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,0.976101,0.859409,708.028352,3186.127582,0.859623,0.859456,1.011868,0.621184,0,0


In [26]:
X_test_real['BOA_RT']=BOA_test
X_test_real['alpha']=y_test
X_test_real['alpha_ai']=y_pred

In [27]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,alpha_ai
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,3187.429121,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.224256,-0.219688
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,3186.056472,0.471695,0.470791,1.873413,1.217586,0,0,594.575497,0.376680,0.376612
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,3185.507886,0.243492,0.241375,4.099723,4.110381,0,0,284.591943,0.238902,0.229414
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,3187.535216,0.454355,0.453397,1.400187,0.288163,0,0,373.718733,0.471005,0.468541
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,3185.812230,0.999159,0.999158,0.933753,0.732430,0,0,832.137702,0.128247,0.126366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,3186.288112,0.962308,0.962266,0.872219,0.472533,0,0,801.588284,0.194530,0.194265
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,3186.551091,0.042399,0.030856,18.672395,11.328534,0,2,32.089238,-0.142424,-0.150260
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,3185.523112,0.909141,0.909037,1.094304,1.074933,0,0,735.627029,0.149952,0.152813
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,3186.127582,0.859623,0.859456,1.011868,0.621184,0,0,924.379368,0.143431,0.140123


In [28]:
def calculate_BOA_IA(Tg_abs,Mg,Ta_abs,Ma,alpha,delta_g_scat,AODS):
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*AODS)*Ma
    
    BOA_ia= numerator / denominator
    return BOA_ia


In [30]:
X_test_real['BOA_AI'] = X_test_real.apply(lambda row: calculate_BOA_IA(row['Tg_abs'], row['mprime_g'],row['Ta_abs'],row['mprime_a'],row['alpha_ai'],row['GOD'],row['AODS']), axis=1)

In [31]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,alpha_ai,BOA_AI
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.224256,-0.219688,914.378481
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.471695,0.470791,1.873413,1.217586,0,0,594.575497,0.376680,0.376612,594.608170
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.243492,0.241375,4.099723,4.110381,0,0,284.591943,0.238902,0.229414,286.792481
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.454355,0.453397,1.400187,0.288163,0,0,373.718733,0.471005,0.468541,373.965582
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999159,0.999158,0.933753,0.732430,0,0,832.137702,0.128247,0.126366,832.591657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962308,0.962266,0.872219,0.472533,0,0,801.588284,0.194530,0.194265,801.607407
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.042399,0.030856,18.672395,11.328534,0,2,32.089238,-0.142424,-0.150260,-0.002811
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909141,0.909037,1.094304,1.074933,0,0,735.627029,0.149952,0.152813,735.353831
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859623,0.859456,1.011868,0.621184,0,0,924.379368,0.143431,0.140123,925.058904


In [32]:
X_test_real[X_test_real['alpha']<0]

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,alpha_ai,BOA_AI
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.224256,-0.219688,914.378481
288612,0.825671,0.564232,0.908966,0.677930,0.191559,0.296357,0.200909,89.409767,1485.490192,0.780309,...,0.032205,0.018693,26.326266,25.453480,1,3,39.535462,-0.148200,-0.146392,0.002068
29128,0.794359,0.557249,0.717551,0.667102,0.230219,0.997035,0.665124,77.131651,1594.095076,0.185534,...,0.225681,0.223379,3.711775,2.017457,0,0,63.202855,-0.058098,-0.056010,63.017495
101886,0.800287,0.864360,0.996605,0.946658,0.222785,0.063752,0.060351,19.725620,2999.999538,0.969139,...,0.941405,0.941339,0.761130,0.237035,0,0,908.012146,-0.086843,-0.089276,908.403260
4598,0.999184,0.750544,0.889975,0.127001,0.000816,0.133519,0.016957,1.237485,1387.595504,0.772061,...,0.999767,0.999767,0.857320,0.499791,0,0,737.771313,-0.042250,-0.021269,737.716746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197139,0.751529,0.209391,0.942473,0.239673,0.285645,0.077924,0.018676,75.441821,4882.544470,0.695266,...,0.253963,0.251946,2.288880,0.345511,0,0,33.382579,-0.275760,-0.199773,31.467737
21471,0.967258,0.929883,0.804170,0.735812,0.033290,0.824958,0.607014,2.832832,2122.014849,0.780631,...,0.998780,0.998778,0.790920,0.346530,0,0,887.156788,-0.136866,-0.134377,886.941060
348457,0.793460,0.856877,0.733558,0.539780,0.231352,0.673262,0.363413,78.531962,0.000000,0.893919,...,0.202172,0.199576,4.946280,5.010612,0,0,120.685924,-0.104428,-0.109200,121.932980
28931,0.395221,0.791149,0.913759,0.213810,0.928309,0.114716,0.024527,83.230227,1259.653337,0.201239,...,0.123511,0.119178,7.039010,4.469645,0,0,166.440440,-0.034727,-0.036721,169.314362


In [33]:
X_test_real['erreur']=np.abs(X_test_real['BOA_AI']-X_test_real['BOA_RT'])

In [35]:
NEG=X_test_real[X_test_real['alpha']<0]
NEG[NEG['erreur']> 12]

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,alpha_ai,BOA_AI,erreur
288612,0.825671,0.564232,0.908966,0.677930,0.191559,0.296357,0.200909,89.409767,1485.490192,0.780309,...,0.018693,26.326266,25.453480,1,3,39.535462,-0.148200,-0.146392,0.002068,39.533394
50071,0.955728,0.832400,0.798153,0.765945,0.045281,0.963254,0.737799,86.327487,184.881798,0.299047,...,0.066407,13.310996,13.729082,0,1,63.785075,-0.235793,-0.229758,45.922777,17.862298
299613,0.739538,0.895520,0.983564,0.948833,0.301730,0.323894,0.307321,87.688035,76.477639,0.646117,...,0.043908,18.532080,21.920310,1,2,252.068447,-0.082055,-0.077444,228.894941,23.173506
110403,0.739652,0.911388,0.892361,0.432927,0.301575,0.200829,0.086944,88.828536,2032.586583,0.187485,...,0.026387,20.629635,13.716432,0,2,259.581780,-0.133084,-0.139267,395.419415,135.837635
198961,0.719408,0.878693,0.831949,0.306886,0.329327,0.265445,0.081461,86.668837,745.985319,0.500678,...,0.060684,13.459695,11.348515,0,1,182.474317,-0.185803,-0.194269,275.164128,92.689811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179842,0.521711,0.795928,0.848983,0.414307,0.650642,0.279526,0.115809,88.354524,55.750683,0.366998,...,0.033408,22.312466,29.110033,1,2,118.301027,-0.063906,-0.055876,0.414965,117.886062
136370,0.944831,0.968445,0.969946,0.679856,0.056749,0.095315,0.064801,89.230032,126.165806,0.602779,...,0.020933,28.903080,44.851119,2,3,291.212828,-0.250714,-0.203978,215.309029,75.903799
162554,0.746438,0.780037,0.798492,0.629725,0.292443,0.607740,0.382709,87.259035,40.696515,0.060567,...,0.050897,16.695924,19.251944,1,2,83.312030,-0.135927,-0.136623,-79.457960,162.769990
309544,0.854662,0.871784,0.973533,0.922430,0.157049,0.345800,0.318976,89.938191,3523.416950,0.005816,...,0.013075,24.942254,13.135969,0,3,224.949095,-0.168995,-0.161260,160.332463,64.616632


In [57]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [58]:
erreur=X_test_real[X_test_real['erreur']> 12]
erreur.drop(columns=['mprime_a_bin','mprime_g_bin'],inplace=True)
erreur.to_excel('data.xlsx',index=False)

In [ ]:
tg_scat, tg_abs, optical_depth, albedo, a, g1=

In [46]:
import math
def muprime(z,h,µ):
    RAYON_TERRESTRE=6371
    eta = (RAYON_TERRESTRE*1000 + z) / h
    root = (eta*µ)**2  + 2 * eta + 1
    sum = (root)**0.5 - eta * µ
    if sum > 0 :
        return 1/sum
    return 1 
data=[]
SZA=50
z=34
reflectance_value=0.44
Hg=9000
Ha=2000
Tg_scat, Tg_abs, AOD, SSA, a, g1  = [0.9641000000000001, 0.8691486434014212, 0.23898261880326274, 0.9405799874037322, 1.0, 0.9511456870141082]
GOD=-math.log(Tg_scat)
µ=math.cos(math.radians(SZA))
Ta_abs=math.exp(-AOD*(1-SSA))
AODS=AOD*SSA
mu_g=(6371*1000+z)/Hg
mu_a=(6371*1000+z)/Ha
muprime_g=muprime(z,Hg,µ)
muprime_a=muprime(z,Ha,µ)
mprime_g=math.exp(-z/Hg)/muprime_g
mprime_a=math.exp(-z/Ha)/muprime_a
bin_g = np.digitize(mprime_g, bins_g)
bin_a = np.digitize(mprime_a, bins_a)
new_l = [Tg_scat,Tg_abs, Ta_abs,SSA,GOD,AOD, AODS,SZA,z,reflectance_value,g1, µ,mu_g, mu_a, muprime_g, muprime_a, mprime_g, mprime_a,bin_g,bin_a]
data.append(new_l)

In [53]:
GOD

0.036560255311802285

In [47]:
X_test_real.columns

Index(['Tg_scat', 'Tg_abs', 'Ta_abs', 'SSA', 'GOD', 'AOD', 'AODS', 'SZA', 'Z',
       'R_scence', 'g1', 'Cos(SZA)', 'mu_g', 'mu_a', 'muprime_g', 'muprime_a',
       'mprime_g', 'mprime_a', 'mprime_a_bin', 'mprime_g_bin', 'BOA_RT',
       'alpha', 'alpha_ai', 'BOA_AI', 'erreur'],
      dtype='object')

In [49]:
columns = [
'Tg_scat','Tg_abs','Ta_abs','SSA','GOD','AOD','AODS','SZA','Z','R_scence','g1','Cos(SZA)','mu_g', 'mu_a', 'muprime_g',
'muprime_a', 'mprime_g','mprime_a', 'mprime_a_bin', 'mprime_g_bin'
]
sample = pd.DataFrame(data, columns=columns)


In [50]:
cols_to_scale = sample.drop(columns=['mprime_a_bin','mprime_g_bin']).columns
cols_to_encode = ['mprime_a_bin', 'mprime_g_bin']
X_scaled = scaler.transform(sample[cols_to_scale])
X_bins_encoded = encoder.transform(sample[cols_to_encode])
encoded_feature_names = encoder.get_feature_names_out(cols_to_encode)

# Construction du DataFrame final
sample_scaled = pd.DataFrame(
    np.concatenate([X_scaled, X_bins_encoded], axis=1),
    columns=list(cols_to_scale) + list(encoded_feature_names),
    index=sample.index  # Pour conserver les bons index
)

In [51]:
sample_scaled

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_0,mprime_a_bin_1,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4
0,0.839555,0.324221,1.342282,1.667113,-0.689605,-0.576425,-0.197806,0.252929,-1.10169,-0.207766,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [52]:
model.predict(sample_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


array([[0.09404449]], dtype=float32)

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [1]:
import tensorflow as tf

2025-05-29 17:54:07.784842: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-29 17:54:07.790027: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-29 17:54:07.807017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748534047.834870   74608 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748534047.841547   74608 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748534047.862404   74608 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin